# Descriptive analysis

Input: data/data_s3.csv

1. Translate (map) numerical levels of variables into readable English version.

2. Generate statistics tables for all the crash records.

3. Generate statistics tables for all the crash records' participants.

4. Process to generate data for Bayesian network analysis.

Output: data/data_s4.csv

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import scipy.stats as stats
import os
from pathlib import Path
import json
import numpy as np
import matplotlib.pyplot as plt

figure_path = os.path.join(Path(os.getcwd()).parent.parent, 'figures/')
data_path = os.path.join(Path(os.getcwd()).parent.parent, 'data/')
with open(os.path.join(data_path, 'value_mapping.txt'), encoding='utf-8') as json_file:
    value_dict = json.load(json_file)

In [3]:
# Load data
df = pd.read_csv(os.path.join(data_path, 'data_s3.csv'), low_memory=False)

## 1 Data summary: variables and levels
### 1.1 Summarize variables and its levels

In [4]:
# Rename columns
var_mapping = {'weekday': 'Day of week',
               'hour': 'Time of day',
               'weather': 'Weather',
               'cluster': 'Land-use cluster',
               'type': 'Crash type',
               'reason': 'Crash causation',
               'road_type': 'Road type',
               'veh_type': 'Vehicle type',
               'injs': 'Injuries (bin)',
               'deaths': 'Deaths (bin)',
               'injs_num': 'Injuries',
               'deaths_num': 'Deaths',
               'age': 'Age',
               'edu': 'Education level',
               'gender': 'Gender',
               'respon': 'Responsibility',
               'travel_mode': 'Travel method'}

df = df.rename(columns={k: v for k, v in var_mapping.items()})

In [5]:
# Extract levels
col_list = list(df.columns)
col_list.remove('id')
level_df_list = []
for var in col_list:
    level_df_list.append(pd.DataFrame([(var, x) for x in df[var].unique()], columns=['var', 'level']))
df_level = pd.concat(level_df_list)
df_level.to_clipboard(index=False)
df.iloc[0]

id                         1
Crash type                11
Weather                    1
Crash causation         1094
Road type                 21
Gender                     1
Age                 (45, 50]
Education level      Unknown
Responsibility             0
Vehicle type               2
Travel method              3
Injuries (bin)           = 0
Deaths (bin)             = 0
Day of week                3
Time of day                0
Injuries                   0
Deaths                     0
Land-use cluster           2
Name: 0, dtype: object

### 1.2 Manually define the translate of levels for each variable
The file is stored in bn_var_level_translate.xlsx.


In [6]:
## Prepare table for write-up
df_var_list = []
df_lvs = pd.read_excel(os.path.join(Path(os.getcwd()).parent.parent, 'docs/var_level_translate.xlsx'))

for var, frame in df_lvs.groupby('var'):
    ld = '; '.join([' '.join([str(row['level_index']) + ')', str(row['en'])]) for _, row in frame.sort_values(by='level_index').iterrows()])
    df_var_list.append(pd.DataFrame([(var, str(len(frame)), ld)], columns=('Variable', 'Level number', 'Level description')))
df_var_tb = pd.concat(df_var_list)
df_var_tb.to_clipboard(index=False)

## 2 Replace numerical levels with English abbreviations for easy description

In [7]:
df_lvs = df_lvs.astype(str)

In [8]:
for var in df.columns:
    if var not in ['id', 'Injuries', 'Deaths']:
        df.loc[:, var] = df.loc[:, var].astype(str)
        lvs_dict = {row['level']: row['abbr'] for _, row in df_lvs.loc[df_lvs['var'] == var, :].iterrows()}
        df.loc[:, var] = df.loc[:, var].apply(lambda x: lvs_dict[x])

In [9]:
for var in df.columns:
    print(var, df[var].unique(), '\n')

id [     1      2      3 ... 237253 237254 237255] 

Crash type ['CT3' 'CT1' 'CT6' 'Others' 'CT5' 'Unknown' 'CT7' 'CT4' 'CT12' 'CT2'
 'CT11' 'CT10' 'CT9' 'CT8' 'CT13'] 

Weather ['Sunny' 'Cloudy' 'Light rain' 'Unknown' 'Heavy rain' 'Haze or fog'] 

Crash causation ['CC8' 'CC2' 'CC9' 'CC10' 'Others' 'Unknown' 'CC7' 'CC14' 'CC5' 'CC13'
 'CC18' 'CC17' 'CC3' 'CC6' 'CC16' 'CC15' 'CC12' 'CC4' 'CC11' 'CC1'] 

Road type ['RT6' 'RT2' 'RT11' 'RT1' 'RT7' 'Unknown' 'RT10' 'RT8' 'RT3' 'RT9' 'RT5'
 'RT4'] 

Gender ['Male' 'Female' 'Unknown'] 

Age ['(45, 50]' '(60, 65]' '(25, 30]' '(18, 25]' '(30, 35]' '(35, 40]'
 '(50, 55]' '(40, 45]' 'Unknown' '(55, 60]' '< 18' '> 70' '(65, 70]'] 

Education level ['Unknown' 'EL3' 'EL4' 'EL6' 'EL2' 'EL5' 'EL1' 'EL7'] 

Responsibility ['No' 'Full' 'Equal' 'Major' 'Minor' 'Unknown'] 

Vehicle type ['Bus' 'Motorcycle' 'Truck' 'Others' 'Car' 'Unknown' 'Non-motor vehicle'] 

Travel method ['Driving motor vehicle' 'Driving non-motor vehicle' 'Walking' 'Unknown'
 'Others

In [10]:
# Remove duplicated records to look at the crash attributes
df_rec = df.drop_duplicates(subset=['id'])


## 3 Statistics of different crash attributes
This part of the results are summarised in docs/article_tables.xlsx.

In [11]:
print(f'Total number of crashes: {len(df_rec)}')
print(f'Total number of the involved traffic participants: {len(df)}')

Total number of crashes: 237255
Total number of the involved traffic participants: 436412


In [12]:
def basic(data):
    data = pd.merge(data, df_lvs.loc[df_lvs['var'] == var, ['en', 'abbr']],
                    left_on=var, right_on='abbr')
    num = len(data)
    injs = data['Injuries'].sum()
    deaths = data['Deaths'].sum()
    return pd.Series({'Var en': data.iloc[0]['en'],
                      'Crash number': num, 'Crash %': num / len(df_rec) * 100,
                      'Deaths number': deaths, 'Deaths %': deaths / df_rec['Deaths'].sum() * 100,
                      'Injuries number': injs, 'Injuries %': injs / df_rec['Injuries'].sum() * 100
                      })

### 3.1 Travel method (Travel method)
Impact on: number of crashes, deaths, injuries.

In [13]:
var = 'Travel method'
df_rec.groupby(var).apply(basic).to_clipboard()
df_rec.groupby(var).apply(basic)

,Var en,Crash number,Crash %,Deaths number,Deaths %,Injuries number,Injuries %
Travel method,,,,,,,
Driving motor vehicle,Driving motor vehicle,154163,64.977767,859,55.241158,103246,56.703335
Driving non-motor vehicle,Driving non-motor vehicle,28216,11.892689,272,17.491961,47684,26.188345
Others,Others,392,0.165223,17,1.093248,105,0.057667
Unknown,Unknown,43893,18.500348,180,11.575563,13376,7.346181
Walking,Walking,10591,4.463973,227,14.598071,17670,9.704472


### 3.2 Road type (Road type)
Impact on: number of crashes, deaths, injuries.

In [14]:
var = 'Road type'
df_rec.groupby(var).apply(basic).to_clipboard()
df_rec.groupby(var).apply(basic)

,Var en,Crash number,Crash %,Deaths number,Deaths %,Injuries number,Injuries %
Road type,,,,,,,
RT1,Highway,39137,16.495754,108,6.945338,4293,2.357742
RT10,Public square,1509,0.636025,11,0.707395,1428,0.784266
RT11,Other road,26005,10.960781,240,15.434084,23812,13.077696
RT2,1st class road,8961,3.776949,67,4.308682,6150,3.377618
RT3,2nd class road,215,0.090620,0,0.000000,68,0.037346
RT4,3rd class road,238,0.100314,3,0.192926,261,0.143343
RT5,4th class road,7,0.002950,0,0.000000,18,0.009886
RT6,City expressway,12360,5.209585,62,3.987138,4081,2.241310
RT7,Normal urban road/street,126047,53.127226,1045,67.202572,130566,71.707647


### 3.3 Crash type (Crash type)
Impact on: number of crashes, deaths, injuries.

In [15]:
var = 'Crash type'
df_rec.groupby(var).apply(basic).to_clipboard()
df_rec.groupby(var).apply(basic)

,Var en,Crash number,Crash %,Deaths number,Deaths %,Injuries number,Injuries %
Crash type,,,,,,,
CT1,Collision with fixed objects,18080,7.620493,10,0.643087,3749,2.058974
CT10,Passenger falling out of vehicles,315,0.132769,0,0.000000,579,0.317990
CT11,Crushing pedestrians,470,0.198099,173,11.125402,559,0.307006
CT12,Rollover,665,0.280289,1,0.064309,650,0.356984
CT13,Other collision type between vehicles and humans,292,0.123074,0,0.000000,214,0.117530
CT2,Collision with non-fixed objects,840,0.354049,0,0.000000,86,0.047232
CT3,Collision with motor vehicles in transport,157715,66.474890,793,50.996785,120561,66.212839
CT4,Collision with stopped vehicles,2103,0.886388,85,5.466238,1382,0.759003
CT5,Other collision type between vehicles,17650,7.439253,2,0.128617,14665,8.054108


### 3.4 Weather (Weather)
Impact on: number of crashes, deaths, injuries.

In [16]:
var = 'Weather'
df_rec.groupby(var).apply(basic).to_clipboard()
df_rec.groupby(var).apply(basic)

,Var en,Crash number,Crash %,Deaths number,Deaths %,Injuries number,Injuries %
Weather,,,,,,,
Cloudy,Cloudy,12869,5.424122,146,9.389068,8022,4.405732
Haze or fog,Haze or fog,102,0.042992,2,0.128617,29,0.015927
Heavy rain,Heavy rain,54,0.022760,0,0.000000,23,0.012632
Light rain,Light rain,27120,11.430739,182,11.704180,15505,8.515441
Sunny,Sunny,197075,83.064635,1225,78.778135,158482,87.039285
Unknown,Unknown,35,0.014752,0,0.000000,20,0.010984


#### 3.4.1 Compare with the overall weather
Look at the weather records downloaded from OpenWeather.

In [17]:
df_weather = pd.read_csv(os.path.join(data_path, 'weather_shenzhen_2014-2016.csv'),
                         usecols=['dt_iso', 'weather_main'])
df_weather.dt_iso = df_weather.dt_iso.apply(lambda x: x.split(' ')[0])
df_weather.groupby('weather_main').size()

weather_main
Clear            3100
Clouds          16244
Haze                5
Mist               22
Rain             6936
Thunderstorm        5
dtype: int64

### 3.5 Crash causation (Crash causation)
Impact on: number of crashes, deaths, injuries.

In [18]:
var = 'Crash causation'
df_rec.groupby(var).apply(basic).to_clipboard()
df_rec.groupby(var).apply(basic)

,Var en,Crash number,Crash %,Deaths number,Deaths %,Injuries number,Injuries %
Crash causation,,,,,,,
CC1,Motor vehicles not driving on the allowed lanes,582,0.245306,0,0.000000,661,0.363025
CC10,Not passing vehicles driving in the opposite d...,5635,2.375082,24,1.543408,6236,3.424849
CC11,Openning or closing doors to obstruct vehicles...,694,0.292512,0,0.000000,1204,0.661244
CC12,Driving in the opposite direction,1538,0.648248,15,0.964630,2817,1.547114
CC13,Not driving on the right-side of a road,1235,0.520537,5,0.321543,2589,1.421895
CC14,Illegal use of dedicated lanes,1288,0.542876,1,0.064309,2776,1.524596
CC15,Not following traffic signals,366,0.154264,10,0.643087,669,0.367419
CC16,Non-motor vehicles not driving on the allowed ...,1672,0.704727,33,2.122186,2827,1.552606
CC17,Illegal crossing of non-motor vehicles on lane...,1226,0.516744,2,0.128617,1602,0.879828


### 3.6 Impact of month of year, day of week, and time of day
Impact on: number of crashes.

Run scr/plot_crash_by_time.R will get the plots.

### 3.7 Land-use cluster
Impact on: number of crashes.

In [19]:
var = 'Land-use cluster'
df_rec.groupby(var).apply(basic).to_clipboard()
df_rec.groupby(var).apply(basic)

,Var en,Crash number,Crash %,Deaths number,Deaths %,Injuries number,Injuries %
Land-use cluster,,,,,,,
LUC1,1,30607,12.900466,178,11.446945,29361,16.125241
LUC2,2,65691,27.687931,469,30.160772,41058,22.549305
LUC3,3,38396,16.183431,310,19.935691,32486,17.841510
LUC4,4,18220,7.679501,75,4.823151,9234,5.071369
LUC5,5,56783,23.933321,386,24.823151,47348,26.003811
LUC6,6,17207,7.252534,119,7.652733,16810,9.232155
Unknown,Unknown,10351,4.362816,18,1.157556,5784,3.176608


In [20]:
# Check the road type of LUC2 vs LUC5
var1 = 'Land-use cluster'
var2 = 'Road type'
df_crs = df_rec.loc[:, [var1, var2]].pivot_table(index=var2, columns=var1, aggfunc=len, fill_value=0)
df_crs.drop(columns=['Unknown'], index=['Unknown'], inplace=True)
df_crs = df_crs/df_crs.sum(axis=0)*100

df_crs

Land-use cluster,LUC1,LUC2,LUC3,LUC4,LUC5,LUC6
Road type,,,,,,
RT1,0.000000,32.877172,16.884813,54.387744,4.727928,0.000000
RT10,0.000000,2.179022,0.000000,0.766025,0.018777,0.000000
RT11,13.350748,8.826398,11.699319,11.141151,17.933456,2.507172
RT2,1.957346,0.610702,0.144505,0.000000,14.983664,0.000000
RT3,0.000000,0.329331,0.002779,0.000000,0.015021,0.000000
RT4,0.529593,0.000000,0.000000,0.000000,0.016899,0.505176
RT5,0.000000,0.000000,0.019453,0.000000,0.000000,0.000000
RT6,0.572533,5.283689,9.176046,0.405542,10.317699,0.143445
RT7,82.945681,49.674665,61.731277,32.522249,51.688009,96.476238


In [21]:
# Check the deaths + injuries of LUC2 - rural high-speed vs LUC5 - urban low-speed
for var in ['Deaths', 'Injuries']:
    var1 = 'Land-use cluster'
    df_crs = df_rec.loc[df_rec[var1].isin(['LUC2', 'LUC5']), [var, var1]]
    conseq_LUC2 = np.mean(df_crs.loc[df_crs[var1] == 'LUC2', var].values)
    conseq_LUC5 = np.mean(df_crs.loc[df_crs[var1] == 'LUC5', var].values)

    print(f"{var}: LUC2 {conseq_LUC2}, LUC5 {conseq_LUC5}")
    print(stats.ks_2samp(df_crs.loc[df_crs[var1] == 'LUC2', var].values,
                         df_crs.loc[df_crs[var1] == 'LUC5', var].values))

Deaths: LUC2 0.007139486383218401, LUC5 0.006797809203458782
Ks_2sampResult(statistic=0.00016049780315907647, pvalue=1.0)
Injuries: LUC2 0.6250171256336484, LUC5 0.8338411144180476
Ks_2sampResult(statistic=0.09735227757096243, pvalue=1.7200415913676988e-251)


### 3.8 Crash type x road type
Impact on: number of crashes. How distribution of crash type differs between different road types.

In [22]:
var1 = 'Crash type'
var2 = 'Road type'
df_crs = df_rec.loc[:, [var1, var2]].pivot_table(index=var2, columns=var1, aggfunc=len, fill_value=0)
df_crs.drop(columns=['Others', 'Unknown'], index=['Unknown'], inplace=True)
df_crs.to_csv(os.path.join(data_path, 'crash_x_road.csv'))

g, p, dof, expctd = stats.chi2_contingency(df_crs)
print(g, p, dof)
df_crs

19558.717114622104 0.0 120


Crash type,CT1,CT10,CT11,CT12,CT13,CT2,CT3,CT4,CT5,CT6,CT7,CT8,CT9
Road type,,,,,,,,,,,,,
RT1,5559,2,12,222,92,476,32438,90,42,59,35,2,19
RT10,135,7,1,10,0,6,1222,11,11,103,0,0,0
RT11,2271,75,70,93,11,49,20285,330,386,2287,8,9,1
RT2,564,13,12,21,3,10,7901,39,94,280,2,0,1
RT3,25,0,0,0,0,1,185,2,1,1,0,0,0
RT4,15,0,1,2,0,0,174,11,2,32,0,0,0
RT5,0,0,0,1,0,0,6,0,0,0,0,0,0
RT6,1198,4,12,36,20,59,9869,101,676,268,3,2,2
RT7,8255,211,352,276,165,234,85171,1493,16314,12833,19,19,31


In [23]:
crs_RT = df_crs.sum(axis=1)

### 3.9 Crash causation x Crash type, Crash causation x road type
Impact on: number of crashes. How distribution of crash type & road type differs between different Crash causation.

In [24]:
var1 = 'Crash causation'
var2 = 'Road type'
df_crs = df_rec.loc[:, [var1, var2]].pivot_table(index=var2, columns=var1, aggfunc=len, fill_value=0)
df_crs.drop(columns=['Others', 'Unknown'], index=['Unknown'], inplace=True)
df_crs.to_csv(os.path.join(data_path, 'causation_x_road.csv'))

g, p, dof, expctd = stats.chi2_contingency(df_crs)
print(g, p, dof)
df_crs

53237.52668483571 0.0 170


Crash causation,CC1,CC10,CC11,CC12,CC13,CC14,CC15,CC16,CC17,CC18,CC2,CC3,CC4,CC5,CC6,CC7,CC8,CC9
Road type,,,,,,,,,,,,,,,,,,
RT1,1,0,2,5,0,0,0,0,0,0,8748,22,0,1,0,462,17471,12236
RT10,0,0,2,25,81,0,2,4,0,2,189,24,4,98,14,35,256,720
RT11,10,119,43,266,381,110,73,160,14,15,2341,194,61,1380,236,653,2234,16165
RT2,4,410,11,103,47,0,6,34,86,2,870,14,6,171,21,36,840,6227
RT3,0,0,0,0,1,0,0,0,0,0,3,0,0,1,0,0,5,204
RT4,0,0,0,3,6,0,0,1,0,0,7,2,0,11,3,9,17,174
RT5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,4
RT6,2,6,17,37,4,31,5,23,4,6,3150,48,5,131,17,78,4866,3286
RT7,564,5100,615,1094,714,1142,278,1443,1122,274,7850,1070,286,6024,1637,2319,10170,67172


In [26]:
var1 = 'Crash causation'
var2 = 'Crash type'
df_crs = df_rec.loc[:, [var1, var2]].pivot_table(index=var2, columns=var1, aggfunc=len, fill_value=0)
df_crs.drop(columns=['Others', 'Unknown'], index=['Unknown', 'Others'], inplace=True)
df_crs.to_csv(os.path.join(data_path, 'causation_x_type.csv'))

g, p, dof, expctd = stats.chi2_contingency(df_crs)
print(g, p, dof)
df_crs

30340.29099434455 0.0 204


Crash causation,CC1,CC10,CC11,CC12,CC13,CC14,CC15,CC16,CC17,CC18,CC2,CC3,CC4,CC5,CC6,CC7,CC8,CC9
Crash type,,,,,,,,,,,,,,,,,,
CT1,16,667,0,2,41,16,3,22,25,0,27,4,0,2,3,193,69,16056
CT10,0,23,1,0,35,6,0,5,23,0,0,0,0,0,0,0,2,189
CT11,0,2,0,0,1,2,0,3,1,11,0,1,1,0,0,29,1,292
CT12,1,19,0,0,12,31,0,14,2,0,2,0,0,0,0,8,1,541
CT13,0,5,2,6,4,5,0,8,0,0,4,1,0,1,0,1,8,219
CT2,0,10,0,0,0,2,0,2,1,0,0,0,0,0,0,4,2,779
CT3,545,4048,584,1332,793,877,268,1149,871,77,22347,1275,348,7411,1882,2380,34579,72183
CT4,4,110,10,3,18,7,1,30,14,1,72,3,2,17,2,168,182,1243
CT5,5,34,84,115,68,93,63,65,8,34,669,71,7,276,32,135,789,2974


## 4 Statistics of different crash attributes: traffic participants
This part of the results are summarised in docs/article_tables.xlsx.
This part only focuses on the traffic participants who were driving a motor vehicle.

In [27]:
# Select motor-vehicle drivers
df = df.loc[df['Travel method'] == 'Driving motor vehicle', :]

print(f'Number of drivers involved in the crashes: {len(df)}')

Number of drivers involved in the crashes: 309194


In [28]:
tbl_head = ['Total', 'Full', 'Full%', 'Major', 'Major%', 'Equal', 'Equal%',
            'Minor', 'Minor%', 'No', 'No%', 'Unknown', 'Unknown%']

def res(data):
    mn_get = df_lvs.loc[df_lvs['var'] == 'Responsibility', ['en', 'abbr']]
    mn_get = mn_get.loc[mn_get['en'] != 'Unable to determine', :]
    data = pd.merge(data, mn_get,
                    left_on='Responsibility', right_on='abbr', how='left')
    num = data.groupby('Responsibility')['id'].size().reset_index()
    r = dict()
    r['Total'] = len(data) / len(df) * 100
    for _, row in num.iterrows():
        r[row['Responsibility']] = row['id']
        r[row['Responsibility']+ '%'] = row['id'] / len(data) * 100
    respon = pd.Series(r)
    for vr in tbl_head:
        if vr not in respon.index:
            respon[vr] = 0
    respon = respon[tbl_head]
    return respon

### 4.1 Gender of involved traffic participants
Impact on: responsibility.

In [29]:
df2test = df.groupby('Gender').apply(res)
df2test.drop(columns=['Total', 'Full%', 'Major%', 'Equal%', 'Minor%', 'No%', 'Unknown', 'Unknown%'],
             index=['Unknown'], inplace=True)
g, p, dof, _ = stats.chi2_contingency(df2test)
print(g, p, dof)

# Prepare data for the paper
df.groupby('Gender').apply(res).to_clipboard()
df.groupby('Gender').apply(res)

7887.720027685953 0.0 4


,Total,Full,Full%,Major,Major%,Equal,Equal%,Minor,Minor%,No,No%,Unknown,Unknown%
Gender,,,,,,,,,,,,,
Female,20.920199,40982.0,63.357244,2668.0,4.124668,3422.0,5.290335,566.0,0.875023,16930.0,26.173397,116.0,0.179333
Male,69.123916,95891.0,44.866114,7451.0,3.486223,11743.0,5.494392,5474.0,2.561211,92674.0,43.360923,494.0,0.231136
Unknown,9.955885,3371.0,10.950849,205.0,0.665952,233.0,0.756911,26.0,0.084462,642.0,2.085567,26306.0,85.456258


### 4.2 Age of involved traffic participants
Impact on: responsibility.

In [30]:
df2test = df.groupby('Age').apply(res)
df2test.drop(columns=['Total', 'Full%', 'Major%', 'Equal%', 'Minor%', 'No%', 'Unknown', 'Unknown%'],
             index=['Unknown'], inplace=True)
g, p, dof, _ = stats.chi2_contingency(df2test)
print(g, p, dof)

df.groupby('Age').apply(res).to_clipboard()
df.groupby('Age').apply(res)

2506.114848547541 0.0 44


,Total,Full,Full%,Major,Major%,Equal,Equal%,Minor,Minor%,No,No%,Unknown,Unknown%
Age,,,,,,,,,,,,,
"(18, 25]",10.938763,17690.0,52.303235,1351.0,3.994441,1712.0,5.061794,738.0,2.182012,12257.0,36.239726,74.0,0.218793
"(25, 30]",18.358700,29256.0,51.539708,2152.0,3.791135,3062.0,5.394264,1196.0,2.106969,20981.0,36.961807,117.0,0.206117
"(30, 35]",19.570561,30149.0,49.823999,2211.0,3.653881,3295.0,5.445291,1243.0,2.054172,23491.0,38.821041,122.0,0.201616
"(35, 40]",15.654896,23652.0,48.863730,1774.0,3.664986,2723.0,5.625568,1034.0,2.136187,19112.0,39.484340,109.0,0.225188
"(40, 45]",13.119595,19568.0,48.238629,1464.0,3.609023,2304.0,5.679773,916.0,2.258104,16231.0,40.012326,82.0,0.202145
"(45, 50]",7.475242,11062.0,47.860511,803.0,3.474235,1319.0,5.706745,514.0,2.223857,9385.0,40.604854,30.0,0.129797
"(50, 55]",2.986475,4448.0,48.169807,310.0,3.357158,517.0,5.598874,228.0,2.469136,3706.0,40.134286,25.0,0.270739
"(55, 60]",1.044975,1400.0,43.330238,102.0,3.156917,204.0,6.313835,73.0,2.259362,1443.0,44.661096,9.0,0.278552
"(60, 65]",0.342503,435.0,41.076487,45.0,4.249292,65.0,6.137866,25.0,2.360718,489.0,46.175637,0.0,0.000000


### 4.3 Education level of involved traffic participants
Impact on: responsibility.

In [31]:
df.groupby('Education level').apply(res).to_clipboard()
df.groupby('Education level').apply(res)

,Total,Full,Full%,Major,Major%,Equal,Equal%,Minor,Minor%,No,No%,Unknown,Unknown%
Education level,,,,,,,,,,,,,
EL1,0.023610,4.0,5.479452,3.0,4.109589,2.0,2.739726,2.0,2.739726,60.0,82.191781,2.0,2.739726
EL2,0.149421,130.0,28.138528,38.0,8.225108,47.0,10.173160,25.0,5.411255,215.0,46.536797,7.0,1.515152
EL3,8.578756,12650.0,47.690858,1381.0,5.206409,1869.0,7.046183,911.0,3.434496,9551.0,36.007540,163.0,0.614515
EL4,5.802829,8699.0,48.484004,917.0,5.110913,1045.0,5.824323,578.0,3.221491,6623.0,36.913388,80.0,0.445881
EL5,0.385195,568.0,47.691016,70.0,5.877414,99.0,8.312343,47.0,3.946264,388.0,32.577666,19.0,1.595298
EL6,7.170256,11365.0,51.262968,574.0,2.589084,879.0,3.964817,300.0,1.353180,9020.0,40.685611,32.0,0.144339
EL7,0.011320,17.0,48.571429,2.0,5.714286,4.0,11.428571,1.0,2.857143,10.0,28.571429,1.0,2.857143
Unknown,77.878613,106811.0,44.357464,7339.0,3.047808,11453.0,4.756308,4202.0,1.745046,84379.0,35.041695,26612.0,11.051679


### 4.4 Vehicle type of involved traffic participants
Impact on: responsibility.

In [32]:
df2test = df.groupby('Vehicle type').apply(res)
df2test.drop(columns=['Total', 'Full%', 'Major%', 'Equal%', 'Minor%', 'No%'], inplace=True)
g, p, dof, _ = stats.chi2_contingency(df2test)
print(g, p, dof)

df.groupby('Vehicle type').apply(res).to_clipboard()
df.groupby('Vehicle type').apply(res)

23879.549125132315 0.0 30


,Total,Full,Full%,Major,Major%,Equal,Equal%,Minor,Minor%,No,No%,Unknown,Unknown%
Vehicle type,,,,,,,,,,,,,
Bus,34.718009,58575.0,54.566542,4193.0,3.906061,5747.0,5.353716,1747.0,1.627448,32889.0,30.638310,4195.0,3.907924
Car,48.961493,63143.0,41.709934,4789.0,3.163437,7530.0,4.974040,3023.0,1.996882,56186.0,37.114396,16715.0,11.041312
Motorcycle,1.605788,790.0,15.911380,397.0,7.995972,457.0,9.204431,600.0,12.084592,1918.0,38.630413,803.0,16.173212
Non-motor vehicle,0.008409,8.0,30.769231,2.0,7.692308,1.0,3.846154,5.0,19.230769,10.0,38.461538,0.0,0.000000
Others,4.508496,1693.0,12.144907,89.0,0.638451,146.0,1.047346,89.0,0.638451,10570.0,75.824964,1353.0,9.705882
Truck,10.197805,16035.0,50.854714,854.0,2.708446,1517.0,4.811138,602.0,1.909232,8673.0,27.506264,3850.0,12.210206


## 5 Prepare data for Bayesian network modelling
1 Focus on motor-vehicle drivers and their crashes like data used in the section 4 of this notebook.

2 Drop Education level because of too many missing values.

3 Merge injuries and deaths for the ease of interpretation.

4 Keep the crash records with clear information, i.e., no unknown and others fields.

In [33]:
# Drop Education level
df2bn = df.drop(columns=['Education level'])

In [34]:
# Merge injuries and deaths
var = 'Injuries and deaths'
df2bn.loc[:, var] = df2bn['Injuries'] + df2bn['Deaths']

In [35]:
# Check the distribution
df2bn_injdeaths = df2bn.groupby(var).size()
df2bn_injdeaths.name = 'count'
df2bn_injdeaths = df2bn_injdeaths.reset_index()
df2bn_injdeaths.loc[:, 'cum_count'] = df2bn_injdeaths['count'].cumsum()
df2bn_injdeaths.loc[:, 'cum_p'] = df2bn_injdeaths['count'].cumsum() / df2bn_injdeaths['count'].sum() * 100
df2bn_injdeaths.loc[:, 'p'] = df2bn_injdeaths['count'] / df2bn_injdeaths['count'].sum() * 100
df2bn_injdeaths.head(10)

,Injuries and deaths,count,cum_count,cum_p,p
0,0,208446,208446,67.415927,67.415927
1,1,13390,221836,71.746541,4.330614
2,2,67195,289031,93.478851,21.732310
3,3,1262,290293,93.887009,0.408158
4,4,13521,303814,98.259992,4.372983
5,5,294,304108,98.355078,0.095086
6,6,3358,307466,99.441128,1.086050
7,7,101,307567,99.473793,0.032666
8,8,840,308407,99.745467,0.271674
9,9,54,308461,99.762932,0.017465


In [36]:
df2bn.loc[:, var] = pd.cut(df2bn[var], bins=[-1, 0, 4, 9, 1000])
cat_dict = {'(-1, 0]': '= 0', '(9, 1000]': '> 9'}
df2bn.loc[:, var] = df2bn.loc[:, var].apply(lambda x: cat_dict[str(x)] if str(x) in cat_dict else str(x))
df2bn.loc[:, var].cat.add_categories("Unknown", inplace=True)
df2bn.loc[:, var].fillna("Unknown", inplace=True)

In [37]:
# Keep the crash records with complete information
df2bn.replace(to_replace = ['Unknown', 'Others'], value = np.nan, inplace = True)
df2bn.dropna(inplace=True)

### 5.1 Check crash type, driver responsibility -> injuries and deaths

In [38]:
df2bn.iloc[0]

id                                         1
Crash type                               CT3
Weather                                Sunny
Crash causation                          CC8
Road type                                RT6
Gender                                  Male
Age                                 (45, 50]
Responsibility                            No
Vehicle type                             Bus
Travel method          Driving motor vehicle
Injuries (bin)                           = 0
Deaths (bin)                             = 0
Day of week                              Wed
Time of day                                0
Injuries                                   0
Deaths                                     0
Land-use cluster                        LUC2
Injuries and deaths                      = 0
Name: 0, dtype: object

In [47]:
var1 = 'Crash type'
var2 = 'Injuries and deaths'
df_crs = df2bn.loc[:, [var1, var2]].pivot_table(index=var2, columns=var1, aggfunc=len, fill_value=0)
df_crs = df_crs / df_crs.sum(axis=0)
df_crs.loc[['(4, 9]', '> 9'], :].sum()

Crash type
CT1     0.006541
CT10    0.000000
CT11    0.000000
CT12    0.027855
CT13    0.000000
CT2     0.000000
CT3     0.015167
CT4     0.027614
CT5     0.017471
CT6     0.005439
CT7     0.000000
CT8     0.000000
CT9     0.000000
dtype: float64

In [46]:
var1 = 'Responsibility'
var2 = 'Injuries and deaths'
df_crs = df2bn.loc[:, [var1, var2]].pivot_table(index=var2, columns=var1, aggfunc=len, fill_value=0)
df_crs = df_crs / df_crs.sum(axis=0)
df_crs.loc[['(4, 9]', '> 9'], :].sum()

Responsibility
Equal    0.014849
Full     0.009962
Major    0.032601
Minor    0.037358
No       0.017301
dtype: float64

In [72]:
# Save it
print(f'Number of crashes/drivers to construct the Bayesian network: {len(df2bn)}')
df2bn.to_csv(os.path.join(data_path, 'data_s4.csv'), index=False)

Number of crashes/drivers to construct the Bayesian network: 235901
